# Library

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, Reshape, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from skimage.metrics import structural_similarity as ssim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import mnist_reader
import tensorflow as tf

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import save_img
from scipy.linalg import sqrtm
from tqdm import tqdm

# Read Data

In [4]:
# Load training data
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')

# Load test data
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

# Display some information about the data
print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")


Training data shape: (60000, 784)
Training labels shape: (60000,)
Test data shape: (10000, 784)
Test labels shape: (10000,)


In [5]:
# Filter labels for classes 0 and 1
train_mask = np.isin(y_train, [0, 1])
test_mask = np.isin(y_test, [0, 1])

X_train_filtered = X_train[train_mask]
y_train_filtered = y_train[train_mask]

X_test_filtered = X_test[test_mask]
y_test_filtered = y_test[test_mask]

# Display some information about the filtered data
print(f"Filtered Training data shape: {X_train_filtered.shape}")
print(f"Filtered Training labels shape: {y_train_filtered.shape}")
print(f"Filtered Test data shape: {X_test_filtered.shape}")
print(f"Filtered Test labels shape: {y_test_filtered.shape}")

Filtered Training data shape: (12000, 784)
Filtered Training labels shape: (12000,)
Filtered Test data shape: (2000, 784)
Filtered Test labels shape: (2000,)


In [6]:
# Create DataFrame for filtered data
df_train_filtered = pd.DataFrame(X_train_filtered)
df_train_filtered['label'] = y_train_filtered

df_test_filtered = pd.DataFrame(X_test_filtered)
df_test_filtered['label'] = y_test_filtered

# Generator

In [7]:
# Define the Generator
def build_generator(noise_dim, label_dim, img_size):
    label_input = layers.Input(shape=(1,))
    label_embedding = layers.Embedding(label_dim, noise_dim)(label_input)
    label_embedding = layers.Flatten()(label_embedding)
    
    noise_input = layers.Input(shape=(noise_dim,))
    combined_input = layers.Concatenate()([noise_input, label_embedding])

    x = layers.Dense(128)(combined_input)
    x = layers.ReLU()(x)
    x = layers.Dense(256)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(512)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(1024)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(img_size * img_size, activation='tanh')(x)
    x = layers.Reshape((img_size, img_size, 1))(x)

    model = models.Model([noise_input, label_input], x)
    return model

# Discriminator

In [8]:
# Define the Discriminator
def build_discriminator(img_size, label_dim):
    img_input = layers.Input(shape=(img_size, img_size, 1))
    label_input = layers.Input(shape=(1,))
    
    label_embedding = layers.Embedding(label_dim, img_size * img_size)(label_input)
    label_embedding = layers.Flatten()(label_embedding)
    label_embedding = layers.Reshape((img_size, img_size, 1))(label_embedding)
    
    combined_input = layers.Concatenate()([img_input, label_embedding])
    
    x = layers.Flatten()(combined_input)
    x = layers.Dense(512)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(1024)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(1024)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(512)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model([img_input, label_input], x)
    return model

# GAN

In [9]:
# Define the GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    noise_input = layers.Input(shape=(noise_dim,))
    label_input = layers.Input(shape=(1,))
    img = generator([noise_input, label_input])
    validity = discriminator([img, label_input])
    model = models.Model([noise_input, label_input], validity)
    return model

In [10]:
# Set parameters
img_size = 28
noise_dim = 100
label_dim = 2
batch_size = 64
n_epochs = 200
lr = 0.0002

In [11]:
# Load training data
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
# Load test data
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

# Filter dataset
train_mask = np.isin(y_train, [0, 1])
test_mask = np.isin(y_test, [0, 1])

X_train_filtered = X_train[train_mask]
y_train_filtered = y_train[train_mask]

X_test_filtered = X_test[test_mask]
y_test_filtered = y_test[test_mask]

# Normalize and reshape data
X_train_filtered = (X_train_filtered.astype(np.float32) - 127.5) / 127.5
X_train_filtered = X_train_filtered.reshape((-1, img_size, img_size, 1))

X_test_filtered = (X_test_filtered.astype(np.float32) - 127.5) / 127.5
X_test_filtered = X_test_filtered.reshape((-1, img_size, img_size, 1))

In [12]:
# Build and compile the GAN
generator = build_generator(noise_dim, label_dim, img_size)
discriminator = build_discriminator(img_size, label_dim)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr), metrics=['accuracy'])
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr))

# Train GAN 

In [14]:
for epoch in range(30):
    for _ in tqdm(range(len(X_train_filtered) // batch_size)):
        # Train Discriminator
        idx = np.random.randint(0, X_train_filtered.shape[0], batch_size)
        real_imgs, labels = X_train_filtered[idx], y_train_filtered[idx]

        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        gen_labels = np.random.randint(0, label_dim, batch_size)
        gen_imgs = generator.predict([noise, gen_labels])

        d_loss_real = discriminator.train_on_batch([real_imgs, labels], np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch([gen_imgs, gen_labels], np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        gen_labels = np.random.randint(0, label_dim, batch_size)
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch([noise, gen_labels], valid_y)
        
    print(f"Epoch {epoch+1}/30 [D loss: {d_loss[0]} | acc: {100*d_loss[1]}] [G loss: {g_loss}]")


  0%|          | 0/187 [00:00<?, ?it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


  1%|          | 1/187 [00:01<03:07,  1.01s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


  1%|          | 2/187 [00:01<02:37,  1.17it/s]

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


 76%|███████▋  | 143/187 [04:21<01:25,  1.94s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


 77%|███████▋  | 144/187 [04:23<01:23,  1.94s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
# Save generated images
noise = np.random.normal(0, 1, (batch_size, noise_dim))
labels = np.random.randint(0, label_dim, batch_size)
gen_imgs = generator.predict([noise, labels])
for i, img in enumerate(gen_imgs[:10]):
    save_img(f"generated_img_{i}.png", img)

# Evaluate FID score
def calculate_fid(real_images, generated_images):
    def calculate_activation_statistics(images, model):
        images = tf.image.resize(images, (299, 299))
        act = model.predict(images)
        mu = np.mean(act, axis=0)
        sigma = np.cov(act, rowvar=False)
        return mu, sigma

    inception_model = tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

    real_images = np.repeat(real_images, 3, axis=-1)
    gen_images = np.repeat(generated_images, 3, axis=-1)

    real_mu, real_sigma = calculate_activation_statistics(real_images, inception_model)
    gen_mu, gen_sigma = calculate_activation_statistics(gen_images, inception_model)

    ssdiff = np.sum((real_mu - gen_mu)**2.0)
    covmean = sqrtm(real_sigma.dot(gen_sigma))

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + np.trace(real_sigma + gen_sigma - 2.0 * covmean)
    return fid

# Calculate FID for test data and generated data
real_images = X_test_filtered[:batch_size]
generated_images = gen_imgs[:batch_size]

fid_score = calculate_fid(real_images, generated_images)
print(f"FID score: {fid_score}")

  0%|          | 0/187 [00:00<?, ?it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


c:\Users\ARNOLDUS P.B\Documents\Semester 4\Deep Learning\FinalTest Deepl learning\env\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
  1%|          | 1/187 [00:02<06:17,  2.03s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


  1%|          | 2/187 [00:02<03:17,  1.07s/it]

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


 21%|██▏       | 40/187 [00:19<01:07,  2.17it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


 21%|██▏       | 40/187 [00:19<01:10,  2.07it/s]


KeyboardInterrupt: 